# Clot Burden Prediction

We perform a univariable regression for each target variable and explanatory variable. We then perform multivariable regressions. For these, we focus on body composition only, cardiopulmonary features only, and then a composite model. For each of these, we perform three forms of feature selection, using (1) recursive feature elimination with cross validation, (2) forward sequential feature selection with cross validation, and (3) backward feature selection with cross validation. For these groups of selected features, we also perform sensitivities controlling for gender, age, and both gender and age.

# Imports

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import pickle
from tqdm.notebook import trange, tqdm

# Custom / Lcoal
from config import model_config
from regression import reg

# Stats
from scipy.stats import shapiro
from sklearn.feature_selection import(
	RFECV, SequentialFeatureSelector,
)
from sklearn.metrics import (
    make_scorer, r2_score
)
from sklearn.linear_model import (
	LinearRegression, LogisticRegression, LassoCV
)
from sklearn.model_selection import (
	train_test_split, cross_val_score,
    RepeatedKFold
)
from sklearn.pipeline import (
	Pipeline
)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Global vars
SEED = 123
TEST_SIZE = 0.25
HEATMAP_COLORS = sns.diverging_palette(h_neg=250, h_pos=359, as_cmap=True)
SIGNIFICANCE_CUTOFF = 0.05
CV_FOLDS = 5
CUSTOM_CV = RepeatedKFold(n_splits=CV_FOLDS, n_repeats=10, random_state=SEED)


# Cov type
# Options:
#  - robust
#  - clustered
COV_TYPE = 'robust'


# Import/Preprocess Data

In [12]:
with open(Path('../data/prediction_data.pkl'), 'rb') as f:
	data = pickle.load(f)
	
X = data.get('X')
y = data.get('y')
body_features = data.get('body_features')
cardio_features = data.get('cardio_features')
control_features = data.get('controls')
all_features = body_features + cardio_features + control_features

print(X.shape)
print(y.shape)
print(body_features)
print(cardio_features)
print(control_features)

(96, 34)
(96, 7)
['volume_visceral_fat', 'density_visceral_fat', 'mass_visceral_fat', 'volume_subcutaneous_fat', 'density_subcutaneous_fat', 'mass_subcutaneous_fat', 'volume_intermuscular_fat', 'density_intermuscular_fat', 'mass_intermuscular_fat', 'volume_muscle', 'density_muscle', 'mass_muscle', 'volume_bone', 'density_bone', 'mass_bone', 'bmi']
['emphysema_volume_950hu', 'lung_volume', 'extrapulmonary_artery_volume', 'extrapulmonary_vein_volume', 'intrapulmonary_artery_volume', 'intrapulmonary_vein_volume', 'artery_vein_ratio', 'bv5', 'bv10', 'pb_larger_10', 'pv_diameter', 'a_diameter', 'pv_a', 'heart_volume', 'airway_volume', 'airway_ratio']
['age', 'gender_cl_Male']


In [13]:
pe_numbers = y.index.str[:-2]
pe_numbers

Index(['PE1', 'PE12', 'PE12', 'PE14', 'PE15', 'PE15', 'PE15', 'PE15', 'PE16',
       'PE16', 'PE17', 'PE18', 'PE18', 'PE18', 'PE18', 'PE19', 'PE2', 'PE20',
       'PE21', 'PE22', 'PE22', 'PE22', 'PE23', 'PE23', 'PE23', 'PE24', 'PE24',
       'PE25', 'PE25', 'PE25', 'PE27', 'PE27', 'PE28', 'PE3', 'PE3', 'PE3',
       'PE31', 'PE32', 'PE32', 'PE32', 'PE32', 'PE33', 'PE34', 'PE34', 'PE35',
       'PE36', 'PE36', 'PE37', 'PE37', 'PE37', 'PE4', 'PE40', 'PE40', 'PE41',
       'PE41', 'PE41', 'PE41', 'PE42', 'PE42', 'PE43', 'PE44', 'PE45', 'PE47',
       'PE48', 'PE48', 'PE49', 'PE5', 'PE5', 'PE51', 'PE51', 'PE51', 'PE52',
       'PE52', 'PE52', 'PE52', 'PE52', 'PE52', 'PE52', 'PE53', 'PE54', 'PE54',
       'PE54', 'PE56', 'PE56', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6', 'PE6',
       'PE7', 'PE8', 'PE8', 'PE8', 'PE9'],
      dtype='object')

# OLS Regression Functions

In [14]:
def get_params(model, X, y):
    """Returns pd.Series of coefs for comparison with statsmodels params."""
    model.fit(X, y)
    coef = pd.Series(model.coef_, index=model.feature_names_in_)
    coef['const'] = model.intercept_
    return coef.sort_values()

In [15]:
def model_residual_correlation(model):
    """Returns measure of correlation."""
    return np.corrcoef(np.arange(len(model.resid)), model.resid)[1, 0]

In [16]:
def fit_model(X, y):
    """Fit statsmodels OLS model with robust SEs and sklearn OLS model."""
    # Fit statsmodels model for pvalues and coef
    if COV_TYPE == 'robust':
        model_sm = sm.OLS(y, X).fit(cov_type='HC3')
    elif COV_TYPE == 'clustered':
        model_sm = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': pe_numbers})
    # Define sklearn model for CV evaluation
    model_sk = LinearRegression(fit_intercept=True, n_jobs=-1)
    # Check that model params match
    sk_model_params = get_params(model_sk, X, y)
    sm_model_params = model_sm.params.sort_values()
    params_match = np.isclose(sk_model_params, sm_model_params, atol=1e-5)
    if not np.all(params_match):
        print(f"Regressions on {y.name} did not match for sklearn and statsmodels. CV scores may differ.")
    return model_sm, model_sk

In [17]:
def store_model_results(model_sm, model_sk, X, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - model_sk: sklearn model for cross validation
        - X: X data.
        - y: y data.
    """
    # Calculate CV scores
    cv_r2_scores = cross_val_score(
        model_sk, X, y, 
        scoring='r2', 
        cv=CV_FOLDS, n_jobs=-1
    )

    cv_mse_scores = cross_val_score(
        model_sk, X, y, 
        scoring='neg_root_mean_squared_error', 
        cv=CV_FOLDS, n_jobs=-1
    )
    
    # Store model results
    model_results = pd.DataFrame(
        {
            'y': y.name,
            'model_dfn': [tuple(X.columns.values)],
            'nobs': model_sm.nobs,
            'shapiro_resid_pvalue': shapiro(model_sm.resid).pvalue,
            'r2_train': model_sk.score(X, y),
            'r2_cv_mean': np.mean(np.maximum(cv_r2_scores, np.zeros_like(cv_r2_scores))),
            'r2_cv_std': np.std(np.maximum(cv_r2_scores, np.zeros_like(cv_r2_scores))),
            'rmse_cv_mean': np.mean(np.abs(cv_mse_scores)),
            'rmse_cv_std': np.std(np.abs(cv_mse_scores)),
            'fpvalue': model_sm.f_pvalue
        }
    )
    # Set model index
    model_results = model_results.set_index(['y', 'model_dfn'])
    return model_results

In [18]:
def store_coef_results(model_sm, y):
    """
    Params:
        - model_sm: statsmodel model for coefs, pvalues, and residuals.
        - y: y data.
    """
    results = pd.DataFrame(
        {
            'model_dfn': [tuple(model_sm.params.index) for _ in range(len(model_sm.params))],
            'coef': model_sm.params, 
            'pval': model_sm.pvalues,
        },
    )
    results['signif'] = results['pval'].apply(reg.add_significance)
    results = results.reset_index(names='x')
    results['y'] = y.name
    results = results.pivot(index=['y', 'model_dfn'], columns=['x'], values=['coef', 'pval', 'signif'])
    results.columns = ['_'.join(idx) for idx in results.columns]
    return results

In [19]:
def combine_model_results(model_sm, model_sk, X, y):
    model_results = store_model_results(model_sm, model_sk, X, y)
    coef_results = store_coef_results(model_sm, y)
    assert model_results.shape[0] == coef_results.shape[0] 
    combined_results = pd.concat([model_results, coef_results], axis=1)
    return combined_results

### Example

In [20]:
target = 'total_clot_burden'
features = 'density_visceral_fat'
X_temp = sm.add_constant(X[features])
y_temp = y.loc[:, target]
model_sm, model_sk = fit_model(X_temp, y_temp)
combine_model_results(model_sm, model_sk, X_temp, y_temp)

,,nobs,shapiro_resid_pvalue,r2_train,r2_cv_mean,r2_cv_std,rmse_cv_mean,rmse_cv_std,fpvalue,coef_const,coef_density_visceral_fat,pval_const,pval_density_visceral_fat,signif_const,signif_density_visceral_fat
y,model_dfn,,,,,,,,,,,,,,
total_clot_burden,"(const, density_visceral_fat)",96.0,1.260388e-08,0.059386,0.020434,0.028032,0.968757,0.15667,0.02186,0.0,-0.243692,1.0,0.019724,,*


# Perform univariable regressions

In [21]:
univariable_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    for feature in all_features:
        # Fit model
        X_temp = sm.add_constant(X[feature])
        y_temp = y.loc[:, target]
        model_sm, model_sk = fit_model(X_temp, y_temp)

        # Store results
        univariable_results = pd.concat(
            [univariable_results, combine_model_results(model_sm, model_sk, X_temp, y_temp)],
            axis=0
        )
        
print(univariable_results.shape)

univariable_results = univariable_results.reset_index()
univariable_results['selection_method'] = 'All'
univariable_results['model_dfn'] = univariable_results['model_dfn'].apply(lambda x: x[1])
univariable_results['category'] = 'univariable_' + univariable_results['model_dfn']
univariable_results['controls'] = 'None'
univariable_results.index = univariable_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariable_results.index.name = 'Lookup'

print(univariable_results.shape)

univariable_results.tail()

  0%|          | 0/7 [00:00<?, ?it/s]

(238, 113)
(238, 118)


,y,model_dfn,nobs,shapiro_resid_pvalue,r2_train,r2_cv_mean,r2_cv_std,rmse_cv_mean,rmse_cv_std,fpvalue,...,signif_airway_ratio,coef_age,pval_age,signif_age,coef_gender_cl_Male,pval_gender_cl_Male,signif_gender_cl_Male,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_heart_volume%All%centralartery%None,centralartery,heart_volume,96.0,1.617280e-12,0.013149,0.008924,0.017848,0.984017,0.248649,0.228043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_heart_volume,None
univariable_airway_volume%All%centralartery%None,centralartery,airway_volume,96.0,9.714120e-11,0.043528,0.048673,0.051142,0.996160,0.215001,0.045155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_airway_volume,None
univariable_airway_ratio%All%centralartery%None,centralartery,airway_ratio,96.0,1.702099e-11,0.020135,0.018192,0.023142,0.994672,0.233481,0.298965,...,,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_airway_ratio,None
univariable_age%All%centralartery%None,centralartery,age,96.0,1.521341e-12,0.012366,0.008137,0.014811,0.978873,0.237884,0.305216,...,NaN,0.111203,0.30257,,NaN,NaN,NaN,All,univariable_age,None
univariable_gender_cl_Male%All%centralartery%None,centralartery,gender_cl_Male,96.0,3.216951e-12,0.019572,0.037609,0.048292,0.965633,0.247260,0.051687,...,NaN,NaN,NaN,NaN,0.426849,0.048746,*,All,univariable_gender_cl_Male,None


## Control for gender

In [22]:
univariable_gender_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    for feature in all_features:

        if feature in model_config.controls_encoded:
            continue
            
        # Fit model
        features = [feature, 'gender_cl_Male']
        X_temp = sm.add_constant(X[features])
        y_temp = y[target]
        model_sm, model_sk = fit_model(X_temp, y_temp)

        # Store results
        univariable_gender_results = pd.concat(
            [univariable_gender_results, combine_model_results(model_sm, model_sk, X_temp, y_temp)],
            axis=0
        )
        
print(univariable_gender_results.shape)

univariable_gender_results = univariable_gender_results.reset_index()
univariable_gender_results['selection_method'] = 'All'
univariable_gender_results['model_dfn'] = univariable_gender_results['model_dfn'].apply(lambda x: x[1])
univariable_gender_results['category'] = 'univariable_' + univariable_gender_results['model_dfn']
univariable_gender_results['controls'] = 'gender'
univariable_gender_results.index = univariable_gender_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariable_gender_results.index.name = 'Lookup'

print(univariable_gender_results.shape)

univariable_gender_results.tail()

  0%|          | 0/7 [00:00<?, ?it/s]

(224, 110)
(224, 115)


,y,model_dfn,nobs,shapiro_resid_pvalue,r2_train,r2_cv_mean,r2_cv_std,rmse_cv_mean,rmse_cv_std,fpvalue,...,signif_heart_volume,coef_airway_volume,pval_airway_volume,signif_airway_volume,coef_airway_ratio,pval_airway_ratio,signif_airway_ratio,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_a_diameter%All%centralartery%gender,centralartery,a_diameter,96.0,9.783634e-12,0.043754,0.047400,0.054479,0.956746,0.248124,0.135715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_a_diameter,gender
univariable_pv_a%All%centralartery%gender,centralartery,pv_a,96.0,3.017444e-11,0.059134,0.029768,0.046506,0.950988,0.243878,0.014849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_pv_a,gender
univariable_heart_volume%All%centralartery%gender,centralartery,heart_volume,96.0,3.470715e-12,0.022395,0.032426,0.045954,0.977892,0.248071,0.152671,...,,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_heart_volume,gender
univariable_airway_volume%All%centralartery%gender,centralartery,airway_volume,96.0,4.232489e-11,0.044432,0.030204,0.052654,1.023234,0.212740,0.085889,...,NaN,0.13358,0.342758,,NaN,NaN,NaN,All,univariable_airway_volume,gender
univariable_airway_ratio%All%centralartery%gender,centralartery,airway_ratio,96.0,5.575437e-11,0.039828,0.037974,0.040843,0.995769,0.235909,0.057399,...,NaN,NaN,NaN,NaN,0.142789,0.271313,,All,univariable_airway_ratio,gender


## Control for age

In [23]:
univariable_age_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    for feature in all_features:

        if feature in model_config.controls_encoded:
            continue
            
        # Fit model
        features = [feature, 'age']
        X_temp = sm.add_constant(X[features])
        y_temp = y[target]
        model_sm, model_sk = fit_model(X_temp, y_temp)

        # Store results
        univariable_age_results = pd.concat(
            [univariable_age_results, combine_model_results(model_sm, model_sk, X_temp, y_temp)],
            axis=0
        )
        
print(univariable_age_results.shape)

univariable_age_results = univariable_age_results.reset_index()
univariable_age_results['selection_method'] = 'All'
univariable_age_results['model_dfn'] = univariable_age_results['model_dfn'].apply(lambda x: x[1])
univariable_age_results['category'] = 'univariable_' + univariable_age_results['model_dfn']
univariable_age_results['controls'] = 'age'
univariable_age_results.index = univariable_age_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
univariable_age_results.index.name = 'Lookup'

print(univariable_age_results.shape)

univariable_age_results.tail()

  0%|          | 0/7 [00:00<?, ?it/s]

(224, 110)
(224, 115)


,y,model_dfn,nobs,shapiro_resid_pvalue,r2_train,r2_cv_mean,r2_cv_std,rmse_cv_mean,rmse_cv_std,fpvalue,...,signif_heart_volume,coef_airway_volume,pval_airway_volume,signif_airway_volume,coef_airway_ratio,pval_airway_ratio,signif_airway_ratio,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
univariable_a_diameter%All%centralartery%age,centralartery,a_diameter,96.0,3.299585e-12,0.035944,0.012162,0.017484,0.965424,0.244572,0.393036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_a_diameter,age
univariable_pv_a%All%centralartery%age,centralartery,pv_a,96.0,8.683484e-12,0.046021,0.007811,0.009577,0.967766,0.236474,0.062874,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_pv_a,age
univariable_heart_volume%All%centralartery%age,centralartery,heart_volume,96.0,3.893628e-12,0.022795,0.016555,0.030143,0.983444,0.240609,0.348842,...,,NaN,NaN,NaN,NaN,NaN,NaN,All,univariable_heart_volume,age
univariable_airway_volume%All%centralartery%age,centralartery,airway_volume,96.0,9.529956e-11,0.044115,0.044351,0.046240,0.999230,0.213907,0.134826,...,NaN,0.19716,0.083315,,NaN,NaN,NaN,All,univariable_airway_volume,age
univariable_airway_ratio%All%centralartery%age,centralartery,airway_ratio,96.0,5.427875e-11,0.035402,0.014699,0.024792,1.028088,0.220924,0.354581,...,NaN,NaN,NaN,NaN,0.152313,0.290823,,All,univariable_airway_ratio,age


# Perform multivariable regressions

In [24]:
MAX_NUM_REGRESSORS = y.shape[0] // 10
print(f"MAX_NUM_REGRESSORS: {MAX_NUM_REGRESSORS}")

multivariable_results = pd.DataFrame()

for target in tqdm(model_config.num_targets):
    
    low_alpha = -3
    high_alpha = 4
    
    lassoCV = LassoCV(
        alphas=np.logspace(low_alpha, high_alpha, 100),
        cv=CUSTOM_CV,
        fit_intercept=True,
        max_iter=100_000,
        tol=0.001,
        n_jobs=-1
    )

    exceeds_max_num_regressors = True
    while exceeds_max_num_regressors:
        lassoCV.fit(X, y[target])
        coefs = pd.DataFrame(
            {'coef': lassoCV.coef_},
            index=lassoCV.feature_names_in_
        )
        remaining_features_lasso = coefs.loc[~np.isclose(coefs['coef'], 0.0), :].index.values

        if len(remaining_features_lasso) > MAX_NUM_REGRESSORS:
            low_alpha += 0.2
            lassoCV.set_params(**{'alphas': np.logspace(low_alpha, high_alpha, 50)})
        else:
            exceeds_max_num_regressors = False
            
    print(f"{target:<20s} alpha={lassoCV.alpha_:.3f}, feats: {remaining_features_lasso}")
    
    # Fit models
    X_temp_lasso = sm.add_constant(X[remaining_features_lasso])
    y_temp = y[target]
    model_sm_lasso, model_sk_lasso = fit_model(X_temp_lasso, y_temp)

    # Collect model/coef information and store
    model_eval = store_model_results(model_sm_lasso, model_sk_lasso, X_temp_lasso, y_temp)
    model_coefs = store_coef_results(model_sm_lasso, y_temp)
    model_results = pd.concat([model_eval, model_coefs], axis=1)
    multivariable_results = pd.concat([multivariable_results, model_results], axis=0)


MAX_NUM_REGRESSORS: 9


  0%|          | 0/7 [00:00<?, ?it/s]

total_clot_burden    alpha=0.158, feats: ['volume_visceral_fat' 'mass_bone' 'extrapulmonary_vein_volume'
 'artery_vein_ratio' 'bv5' 'pv_a']
superior_right       alpha=0.095, feats: ['volume_visceral_fat' 'volume_bone' 'emphysema_volume_950hu'
 'artery_vein_ratio' 'bv5' 'pb_larger_10' 'pv_a' 'heart_volume']
superior_left        alpha=0.158, feats: ['volume_visceral_fat' 'density_visceral_fat' 'density_bone' 'mass_bone'
 'artery_vein_ratio' 'bv10' 'pv_diameter']
middle_right         alpha=0.081, feats: ['volume_bone' 'emphysema_volume_950hu' 'artery_vein_ratio' 'bv10'
 'pb_larger_10' 'pv_a' 'heart_volume']
inferior_right       alpha=0.158, feats: ['volume_visceral_fat' 'volume_bone' 'emphysema_volume_950hu'
 'extrapulmonary_artery_volume' 'artery_vein_ratio' 'a_diameter' 'pv_a'
 'heart_volume']
inferior_left        alpha=0.158, feats: ['volume_visceral_fat' 'mass_muscle' 'volume_bone'
 'extrapulmonary_artery_volume' 'artery_vein_ratio' 'bv10' 'pv_a'
 'heart_volume']
centralartery        

In [25]:
multivariable_results = multivariable_results.reset_index()
multivariable_results['selection_method'] = 'LassoCV'
multivariable_results['category'] = 'composite'
multivariable_results['controls'] = 'None'
multivariable_results.index = multivariable_results[['category', 'selection_method', 'y', 'controls']].apply('%'.join, axis=1)
multivariable_results.index.name = 'Lookup'
print(multivariable_results.shape)
multivariable_results.head()

(7, 76)


,y,model_dfn,nobs,shapiro_resid_pvalue,r2_train,r2_cv_mean,r2_cv_std,rmse_cv_mean,rmse_cv_std,fpvalue,...,coef_density_intermuscular_fat,pval_airway_ratio,pval_bmi,pval_density_intermuscular_fat,signif_airway_ratio,signif_bmi,signif_density_intermuscular_fat,selection_method,category,controls
Lookup,,,,,,,,,,,,,,,,,,,,,
composite%LassoCV%total_clot_burden%None,total_clot_burden,"(const, volume_visceral_fat, mass_bone, extrap...",96.0,2.881861e-04,0.488691,0.323796,0.179416,0.787585,0.107130,2.380841e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LassoCV,composite,None
composite%LassoCV%superior_right%None,superior_right,"(const, volume_visceral_fat, volume_bone, emph...",96.0,9.218531e-03,0.504607,0.277737,0.156825,0.814730,0.114519,4.729784e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LassoCV,composite,None
composite%LassoCV%superior_left%None,superior_left,"(const, volume_visceral_fat, density_visceral_...",96.0,1.118983e-10,0.375453,0.202938,0.218355,0.806835,0.339188,1.179829e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LassoCV,composite,None
composite%LassoCV%middle_right%None,middle_right,"(const, volume_bone, emphysema_volume_950hu, a...",96.0,2.747475e-08,0.397020,0.278845,0.220811,0.820779,0.161230,4.243319e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LassoCV,composite,None
composite%LassoCV%inferior_right%None,inferior_right,"(const, volume_visceral_fat, volume_bone, emph...",96.0,1.407108e-06,0.467483,0.249573,0.205462,0.871313,0.191928,1.622525e-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LassoCV,composite,None


# Combine Univariate and Multivariable regression results

In [26]:
ols_results = pd.concat(
    [
        univariable_results, 
        univariable_gender_results,
        univariable_age_results, 
        multivariable_results
    ], axis=0
)
if COV_TYPE == 'robust':
    ols_results.to_csv('../output/regressions/ols_results_robust.csv')
elif COV_TYPE == 'clustered':
    ols_results.to_csv('../output/regressions/ols_results_clustered.csv')